In [10]:
import os
from pandas import HDFStore
from tqdm import tqdm
import logging
import time
import pickle
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from collections import defaultdict, Counter
from tqdm import tqdm
import pickle
from queue import Queue
from threading import Thread
matplotlib.use('Agg')
logging.basicConfig(level=logging.INFO)
os.listdir()

['Dock',
 'create_zinc20_hdf5.py',
 'utils.py',
 'DrugAI_drug-likeness.ipynb',
 'dataset',
 'hpc_jp',
 '.gitignore',
 'right_10k.pdbqt',
 'dock_statistics.ipynb',
 '__pycache__',
 'dataset_info.pkl',
 'package.py',
 'create_total_dataset_hdf5.ipynb',
 'create_dataset_from_pdbqt.ipynb',
 'drug_like_100k_smiles_dataset.txt',
 'test.ipynb',
 'repeat_dock_10k.png',
 'dock_statistics_gz.ipynb',
 'random_sample.ipynb',
 'test.png',
 'random_sample.pdbqt',
 'run.py',
 'create_dataset_from_not_pdbqt_format.ipynb',
 'test.py',
 'autodock_vina_batch_dock.py',
 'fpocket',
 'statistics_bind_ratio.ipynb',
 'repeat_dock.ipynb',
 'right_10k_small_pocket.pdbqt',
 'fpocket_filter.ipynb',
 'batch_dock_job_.py',
 '__init__.py',
 'zinc']

In [2]:
# ZINC_DATA_PATH = '../data/dataset/ZINC20-test/BA/'
# ZINC_HDF5_PATH = '../data/dataset/ZINC20-test.h5'

# ZINC_DATA_PATH = 'ZINC-DrugLike-3D-20230407'
# ZINC_DATA_PATH = 'test'
# ZINC_HDF5_PATH = 'ZINC20-DrugLike-3D-20230402.h5'


# 构建h5数据集

In [3]:

from dataset.create_total_dataset_hdf5 import create_total_dataset_hdf5
# from create_total_dataset_hdf5 import create_total_dataset_hdf5
import logging
# logging.basicConfig(level=logging.INFO)

In [ ]:
create_total_dataset_hdf5(ZINC_DATA_PATH)
print('Construct Done! Good Job!')

In [5]:
store = HDFStore('../data/dataset/ZINC20-test/BA_index.h5')
store.keys()

['/BA/EDRP/BAEDRP_xaa',
 '/BA/EDRO/BAEDRO_xaa',
 '/BA/EDRN/BAEDRN_xaa',
 '/BA/EDRM/BAEDRM_xaa',
 '/BA/EDRL/BAEDRL_xaa',
 '/BA/EDMP/BAEDMP_xaa',
 '/BA/EDMO/BAEDMO_xaa',
 '/BA/EDMN/BAEDMN_xaa',
 '/BA/EDMM/BAEDMM_xaa',
 '/BA/EDML/BAEDML_xaa',
 '/BA/EBRP/BAEBRP_xaa',
 '/BA/EBRO/BAEBRO_xaa',
 '/BA/EBRN/BAEBRN_xaa',
 '/BA/EBRM/BAEBRM_xaa',
 '/BA/EBRL/BAEBRL_xaa',
 '/BA/EBMP/BAEBMP_xaa',
 '/BA/EBMO/BAEBMO_xaa',
 '/BA/EBMN/BAEBMN_xaa',
 '/BA/EBMM/BAEBMM_xaa',
 '/BA/EBML/BAEBML_xaa',
 '/BA/EARP/BAEARP_xaa',
 '/BA/EARO/BAEARO_xaa',
 '/BA/EARN/BAEARN_xaa',
 '/BA/EARM/BAEARM_xaa',
 '/BA/EARL/BAEARL_xaa',
 '/BA/EAMO/BAEAMO_xaa',
 '/BA/EAMN/BAEAMN_xaa',
 '/BA/EAMM/BAEAMM_xaa',
 '/BA/CDRN/BACDRN_xaa',
 '/BA/CBRO/BACBRO_xaa',
 '/BA/CBRN/BACBRN_xaa',
 '/BA/CBRM/BACBRM_xaa',
 '/BA/CBRL/BACBRL_xaa',
 '/BA/CBMN/BACBMN_xaa',
 '/BA/CBMM/BACBMM_xaa',
 '/BA/CARO/BACARO_xaa',
 '/BA/CARN/BACARN_xaa',
 '/BA/CARM/BACARM_xaa',
 '/BA/CARL/BACARL_xaa',
 '/BA/CAMO/BACAMO_xaa',
 '/BA/CAMN/BACAMN_xaa',
 '/BA/CAMM/BACAM

# 筛选缺失的数据

In [12]:
# 已下载的数据
total_file = []
for _, _, files in os.walk(ZINC_DATA_PATH):
    total_file.extend(files)
total_file = [file for file in total_file if not file.endswith('.h5')]

In [32]:
# 需要下载的数据
with open('ZINC-downloader-3D-pdbqt.gz.curl', 'r') as f:
    need_download_cmd = f.readlines()
need_download = {file.split(' ')[-2].split('/')[-1]: file for file in need_download_cmd}

In [33]:
# 验证已下载的数据包含于需要下载的数据
total_file_set = set(total_file)
need_download_set = set(need_download.keys())
assert total_file_set.issubset(need_download_set), 'total_file_set is not subset of need_download_set'
not_download = need_download_set - total_file_set

In [35]:
# 生成下载命令
download_cmd = []
for file in not_download:
    download_cmd.append(need_download[file])
len(download_cmd)
download_cmd[:5]

['curl --remote-time --fail --create-dirs -o DC/ADRN/DCADRN.xlu.pdbqt.gz http://files.docking.org/3D/DC/ADRN/DCADRN.xlu.pdbqt.gz\n',
 'curl --remote-time --fail --create-dirs -o FC/AARL/FCAARL.xaa.pdbqt.gz http://files.docking.org/3D/FC/AARL/FCAARL.xaa.pdbqt.gz\n',
 'curl --remote-time --fail --create-dirs -o DC/BCRN/DCBCRN.xaa.pdbqt.gz http://files.docking.org/3D/DC/BCRN/DCBCRN.xaa.pdbqt.gz\n',
 'curl --remote-time --fail --create-dirs -o DC/ADRN/DCADRN.xvv.pdbqt.gz http://files.docking.org/3D/DC/ADRN/DCADRN.xvv.pdbqt.gz\n',
 'curl --remote-time --fail --create-dirs -o DC/ADRN/DCADRN.xfw.pdbqt.gz http://files.docking.org/3D/DC/ADRN/DCADRN.xfw.pdbqt.gz\n']

# 获取数据集的统计信息

In [3]:
hdf_index_file = [os.path.join(ZINC_DATA_PATH, i) for i in os.listdir(ZINC_DATA_PATH) if i.endswith('_index.h5')]
hdf_coor_file = [os.path.join(ZINC_DATA_PATH, i) for i in os.listdir(ZINC_DATA_PATH) if i.endswith('_coor.h5')]
hdf_index_file.sort()
hdf_coor_file.sort()
all_hdf_file = list(zip(hdf_index_file, hdf_coor_file))
all_hdf_file[:5]

[('ZINC-DrugLike-3D-20230407/BA_index.h5',
  'ZINC-DrugLike-3D-20230407/BA_coor.h5'),
 ('ZINC-DrugLike-3D-20230407/BB_index.h5',
  'ZINC-DrugLike-3D-20230407/BB_coor.h5'),
 ('ZINC-DrugLike-3D-20230407/BC_index.h5',
  'ZINC-DrugLike-3D-20230407/BC_coor.h5'),
 ('ZINC-DrugLike-3D-20230407/BD_index.h5',
  'ZINC-DrugLike-3D-20230407/BD_coor.h5'),
 ('ZINC-DrugLike-3D-20230407/BE_index.h5',
  'ZINC-DrugLike-3D-20230407/BE_coor.h5')]

In [10]:
dataset_info = dict()
for index_file, coor_file in tqdm(all_hdf_file):
    index_store = HDFStore(index_file)
    coor_store = HDFStore(coor_file)
    total_molecule = 0
    total_atom = defaultdict(lambda: 0)
    for k in index_store.keys():
        total_molecule += index_store[k].shape[0]
        atom_num_tmp = Counter(coor_store[k]['atom'].to_list())
        for atom, num in atom_num_tmp.items():
            total_atom[atom] += num
    dataset_info[os.path.basename(index_store._path)] = {'total_molecule': total_molecule, 'total_atom': dict(total_atom)}
    index_store.close()
    coor_store.close()
with open('zinc20_druglike_dataset_info.pkl', 'wb') as f:
    pickle.dump(dataset_info, f)
print('All work completed')

100%|██████████| 90/90 [3:32:05<00:00, 141.40s/it]  

All work completed


In [9]:
total_file = 0
total_molecule = 0
for index_file, _ in tqdm(all_hdf_file):
    # 计算时间
    t1 = time.time()
    index_store = HDFStore(index_file)
    t2 = time.time()
    # print(index_store.keys())
    total_file += len(index_store.keys())
    for k in index_store.keys():
        total_molecule += index_store[k].shape[0]
    t3 = time.time()
    logging.info(f'\nfile: {index_file}, keys: {len(index_store.keys())}, time: {t2-t1} s, {t3-t2} s')
    index_store.close()
total_file, total_molecule

100%|██████████| 90/90 [11:00<00:00,  7.33s/it]


(17745, 583632480)

## 获取总分子个数

In [6]:

# 创建两个线程，一个线程用于读取数据，一个线程用于处理数据

def read_data(q: Queue, file_list: list):
    for index_file, coor_file in file_list:
        logging.info(f'index_file: {index_file}, coor_file: {coor_file}')
        q.put((HDFStore(index_file, mode='r'), HDFStore(coor_file, mode='r')))

def process_data(index_store, coor_store):
    logging.info('process_data is running')
    total_molecule_num = 0
    total_atom = defaultdict(lambda: 0)
    for path, sub_group, datasetes in tqdm(index_store.walk()):
        for dataset in datasetes:
            d = os.path.join(path, dataset)
            data = index_store.get(d)
            data: pd.DataFrame
            total_molecule_num += len(data)
            data = coor_store.get(d)
            data: pd.DataFrame
            # 统计原子数量
            atom_num_tmp = Counter(data['atom'].to_list())
            # 统计原子总数
            for key in atom_num_tmp.keys():
                total_atom[key] += atom_num_tmp[key]
    index_store.close()
    coor_store.close()
    return {'total_molecule_num': total_molecule_num, 'total_atom': dict(total_atom)}

# 创建生产者线程类
class Producer(Thread):
    def __init__(self, func, q: Queue, file_list: list):
        super().__init__()
        self.q = q
        self.file_list = file_list
        self.func = func
        self.daemon = True

    def run(self):
        self.func(self.q, self.file_list)
    
# 创建消费者线程类
class Consumer(Thread):
    def __init__(self, q: Queue, loops: int):
        super().__init__()
        self.q = q
        self.daemon = True
        self.loops = loops

    def run(self):
        self.results = dict()
        for i in range(self.loops):
            logging.info('Consumer is running')
            index_store, coor_store = self.q.get()
            logging.info('Consumer get data')
            result  = process_data(index_store, coor_store)
            self.results[os.path.basename(index_store._path)] = result
            self.q.task_done()



### 多线程

In [7]:
q = Queue()
producer = Producer(read_data, q, all_hdf_file)
consumer = Consumer(q, len(all_hdf_file))
producer.start()
consumer.start()
q.join()
producer.join()
consumer.join()
with open('zinc20_druglike_dataset_info.pkl', 'wb') as f:
    pickle.dump(consumer.results, f)
print('All work completed')

INFO:root:index_file: ZINC-DrugLike-3D-20230407/BA_index.h5, coor_file: ZINC-DrugLike-3D-20230407/BA_coor.h5
INFO:root:Consumer is running
INFO:root:index_file: ZINC-DrugLike-3D-20230407/BB_index.h5, coor_file: ZINC-DrugLike-3D-20230407/BB_coor.h5
INFO:root:Consumer get data
INFO:root:process_data is running
2it [00:00, 10.79it/s]INFO:root:index_file: ZINC-DrugLike-3D-20230407/BC_index.h5, coor_file: ZINC-DrugLike-3D-20230407/BC_coor.h5
INFO:root:index_file: ZINC-DrugLike-3D-20230407/BD_index.h5, coor_file: ZINC-DrugLike-3D-20230407/BD_coor.h5
INFO:root:index_file: ZINC-DrugLike-3D-20230407/BE_index.h5, coor_file: ZINC-DrugLike-3D-20230407/BE_coor.h5
4it [00:00,  9.16it/s]INFO:root:index_file: ZINC-DrugLike-3D-20230407/BF_index.h5, coor_file: ZINC-DrugLike-3D-20230407/BF_coor.h5
5it [00:00,  7.95it/s]INFO:root:index_file: ZINC-DrugLike-3D-20230407/BG_index.h5, coor_file: ZINC-DrugLike-3D-20230407/BG_coor.h5
INFO:root:index_file: ZINC-DrugLike-3D-20230407/BH_index.h5, coor_file: ZINC-Dr

All work completed


In [9]:
len(consumer.results.keys())


90

In [ ]:

for index_file, coor_file in tqdm(all_hdf_file):
    print(index_file, coor_file)
    index_file = os.path.join(ZINC_DATA_PATH, index_file)
    coor_file = os.path.join(ZINC_DATA_PATH, coor_file)


In [ ]:

store = HDFStore(ZINC_HDF5_PATH)
# 遍历store
total_molecule_num = 0
total_atom = defaultdict(0)
for path, sub_group, datasetes in tqdm(store.walk()):
    for dataset in datasetes:
        if (d := os.path.join(path, dataset)).endswith('_index'):
            data = store.get(d)
            data: pd.DataFrame
            total_molecule_num += len(data)
        if d.endswith('_coor'):
            data = store.get(d)
            data: pd.DataFrame
            # 统计原子数量
            atom_num_tmp = Counter(data['atom'].to_list())
            # 统计原子总数
            for key in atom_num_tmp.keys():
                total_atom[key] += atom_num_tmp[key]
store.close()
total_atom = dict(total_atom)
dataset_info = {'total_molecule_num': total_molecule_num, 'total_atom': total_atom}
# 保存total_molecule_num, total_atom到文件
with open('dataset_info.pkl', 'wb') as f:
    pickle.dump(dataset_info, f)


# k

# 绘图

In [4]:
dataset_info = pickle.load(open('../data/dataset/zinc20_druglike_dataset_info.pkl', 'rb'))

In [29]:
total_molecule_num = 0
total_atom_count = defaultdict(lambda: 0)
for k, v in dataset_info.items():
    total_molecule_num += v['total_molecule']
    for atom, num in v['total_atom'].items():
        total_atom_count[atom] += num


In [41]:
total_atom_count = dict(total_atom_count)

# 归一化
total_atom_num = sum(total_atom_count.values())
total_atom_count_p = {k: v/total_atom_num for k, v in sorted(total_atom_count.items(), key=lambda item: item[1], reverse=True) if k not in ['HD','HH','HE']}
# 绘制漂亮的条形图
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(total_atom_count_p.keys(), total_atom_count_p.values())
ax.set_xlabel('Atom')
ax.set_ylabel('Atom Count')
ax.set_title('ZINC20 Druglike Subset Atom Count')

# 保存为无背景的图片
plt.savefig('zinc20_druglike_atom_count.png')

In [27]:
total_atom_count, total_atom

({'N': 1830076749,
  'H': 977226094,
  'C': 9736143526,
  'O': 1422815179,
  'F': 210163373,
  'S': 146447751,
  'CL': 57476033,
  'P': 104480,
  'BR': 19456606,
  'HD': 2280,
  'HH': 1460,
  'HE': 934,
  'SI': 108596,
  'I': 1201601},
 [('C', 0.6760635817098539),
  ('N', 0.12707785566521704),
  ('O', 0.09879820726318726),
  ('H', 0.06785715221696192),
  ('F', 0.014593437567469565),
  ('S', 0.010169117865817793),
  ('CL', 0.003991051757678635),
  ('BR', 0.0013510382940792151),
  ('I', 8.34374178725662e-05),
  ('SI', 7.540747578679778e-06),
  ('P', 7.254938552252967e-06),
  ('HD', 1.583198688661635e-07),
  ('HH', 1.0138026690552576e-07),
  ('HE', 6.485559540394593e-08)])